# Tags

In [1]:
%run coco.py --mode=noun --output=data/coco10_tags_nouns.json --html

ImportError: No module named 'spacy'

In [5]:
import json
data = json.load(open('data/coco_noun.tags'))

In [7]:
id = '384029'
data['train2014'][id]['captions']

['A man preparing desserts in a kitchen covered in frosting.',
 'A chef is preparing and decorating many small pastries.',
 'A baker prepares various types of baked goods.',
 'a close up of a person grabbing a pastry in a container',
 'Close up of a hand touching various pastries.']

In [2]:
print(data['train2014'].keys())
id = '384029'
data['train2014'][id]['captions']

dict_keys(['57870', '384029', '222016', '520950', '69675', '547471', '122688', '392136', '398494', '90570'])


['A man preparing desserts in a kitchen covered in frosting.',
 'A chef is preparing and decorating many small pastries.',
 'A baker prepares various types of baked goods.',
 'a close up of a person grabbing a pastry in a container',
 'Close up of a hand touching various pastries.']

In [3]:
item = data['train2014'][id]
print(item['tags'])
print(item['scores'])
cs = item['captions']
cs

['pastry', 'close', 'man', 'chef', 'baker', 'dessert', 'person', 'hand', 'type', 'kitchen', 'good', 'frosting', 'container']
[2.973333333333333, 0.9991666666666666, 0.9990909090909091, 0.999, 0.9988888888888889, 0.9972727272727273, 0.9958333333333333, 0.9955555555555555, 0.9955555555555555, 0.9945454545454545, 0.9922222222222222, 0.9918181818181818, 0.9908333333333333]


['A man preparing desserts in a kitchen covered in frosting.',
 'A chef is preparing and decorating many small pastries.',
 'A baker prepares various types of baked goods.',
 'a close up of a person grabbing a pastry in a container',
 'Close up of a hand touching various pastries.']

In [4]:
# https://spacy.io/api/token#attributes
doc = extractor.nlp(cs[1])
for token in doc:
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
    print(token.text, token.pos_, token.tag_, token.dep_)

A DET DT det
chef NOUN NN nsubj
is VERB VBZ aux
preparing VERB VBG ROOT
and CCONJ CC cc
decorating VERB VBG conj
many ADJ JJ amod
small ADJ JJ amod
pastries NOUN NNS dobj
. PUNCT . punct


In [5]:
assert doc.is_parsed
for token in doc:
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
    print('{}. {} -> {} ({} {})'.format(token.i, token.text, token.head, token.head.i, token.dep_))

0. A -> chef (1 det)
1. chef -> preparing (3 nsubj)
2. is -> preparing (3 aux)
3. preparing -> preparing (3 ROOT)
4. and -> preparing (3 cc)
5. decorating -> preparing (3 conj)
6. many -> pastries (8 amod)
7. small -> pastries (8 amod)
8. pastries -> decorating (5 dobj)
9. . -> preparing (3 punct)


In [87]:
# https://spacy.io/api/span#attributes
sent = list(doc.sents)[0]
sent.root

man

In [88]:
rlocs = [len(list(w.ancestors)) for w in doc]
max(rlocs)

6

## Embeddings Based Merging

### Spacy vectors

In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

tags = item['tags']
vectors = np.array([data['vectors'][tag] for tag in tags])
vectors.shape
S = cosine_similarity(vectors)
for i, j in np.argwhere(S > 0.5):
   print((tags[i], tags[j]))

('pastry', 'pastry')
('pastry', 'frosting')
('close', 'close')
('man', 'man')
('man', 'person')
('man', 'good')
('chef', 'chef')
('chef', 'baker')
('chef', 'kitchen')
('baker', 'chef')
('baker', 'baker')
('dessert', 'dessert')
('dessert', 'frosting')
('person', 'man')
('person', 'person')
('hand', 'hand')
('type', 'type')
('kitchen', 'chef')
('kitchen', 'kitchen')
('good', 'man')
('good', 'good')
('frosting', 'pastry')
('frosting', 'dessert')
('frosting', 'frosting')
('container', 'container')


### FastText vectors

In [7]:
from fastText import load_model
filename = '/home/francolq/fastText/wiki.en/wiki.en.bin'
m = load_model(filename)

In [18]:
vectors = np.array([m.get_word_vector(tag) for tag in tags])
vectors.shape
S = cosine_similarity(vectors)
for i, j in np.argwhere(S > 0.6):
   print((tags[i], tags[j]))

('pastry', 'pastry')
('pastry', 'dessert')
('close', 'close')
('man', 'man')
('chef', 'chef')
('baker', 'baker')
('dessert', 'pastry')
('dessert', 'dessert')
('person', 'person')
('hand', 'hand')
('type', 'type')
('kitchen', 'kitchen')
('good', 'good')
('frosting', 'frosting')
('container', 'container')


### Intersentence

In [53]:
docs = [extractor.nlp(caption) for caption in cs]
sents = [list(doc.sents)[0] for doc in docs]
s1, s2 = sents[:2]
s1 = [token for token in s1 if token.pos_ in ['NOUN', 'VERB']]
s2 = [token for token in s2 if token.pos_ in ['NOUN', 'VERB']]
v1 = np.array([m.get_word_vector(token.text) for token in s1])
v2 = np.array([m.get_word_vector(token.text) for token in s2])
v1.shape, v2.shape
S = cosine_similarity(v1, v2)
i = S.argmax()
i, j = np.unravel_index(i, S.shape)
print(sents[:2])
print(s1[i].i, s2[j].i)

[A man preparing desserts in a kitchen covered in frosting., A chef is preparing and decorating many small pastries.]
2 3


preparing

In [66]:
#def merge(token1, token2):
#     
def tree_reconfigure(new_root):
    # 
    head = new_root.head
    new_children = []
    for child in head.children:
        if child == new_root:
            
        else:
            new_children.append(child)
token1 = s1[i]
list(token1.subtree), list(token1.ancestors)
head = token1.head
list(head.children)

[A, preparing, .]

## Syntactic Merging


- https://spacy.io/api/annotation
- http://universaldependencies.org/u/dep/


In [6]:
docs = [extractor.nlp(caption) for caption in cs]
sents = [list(doc.sents)[0] for doc in docs]
s1, s2 = sents[:2]

In [7]:
print(tags)

NameError: name 'tags' is not defined

In [8]:
# https://spacy.io/api/top-level#options
from spacy import displacy
for doc in docs:
    displacy.render(doc, style='dep', jupyter=True, options={'compact': True, 'distance': 60})

In [131]:
displacy.render(docs[1], style='dep', jupyter=True, options={'compact': False})

## Embeddings

In [66]:
s1

A man with a red helmet on a small moped on a dirt road.